---

---

# Predição sonar (rocha ou mina)

Este notebook implementa uma rede neural para predizer se um objeto é uma mina ou uma rocha com base em sinais de um sonar obtidos em diferentes ângulos.

Trata-se de um problema de classificação binária (R-rocha ou M-mina).

Obs: este notebook utiliza a ferramenta de visualização **TensorBoard**.



---



---




## Conjunto de dados

- Fonte: https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks)

O conjunto de dados contém sinais obtidos de uma variedade de ângulos  diferentes. Cada padrão é um conjunto de 60 números no intervalo de 0,0 a 1,0. Cada número representa a energia dentro de uma determinada banda de frequência, integrada ao longo de um determinado período de tempo.

Detalhes sobre o conjunto de dados:

1. Número de instâncias: 208

2. Número de atributos: 60 

4. Variável target (classe): o rótulo associado a cada registro contém a letra “R” se o objeto for uma rocha e “M” se for uma mina (cilindro de metal)

 

## Leitura e preparação dos dados

Vamos começar importando o arquivo CSV bruto usando o Pandas.

In [ ]:
import pandas as pd

# Clone do repositório de dados do GitHub
!git clone https://github.com/malegopc/AM2PUCPOC
# lê arquivo de dados, atribue NaN para dados faltantes e informa cabeçalho como inexistentes (sem nomes para os atributos)
sonar = pd.read_csv('/content/AM2PUCPOC/Datasets/Sonar/sonar.all-data.csv', na_values=['?'], header = None)
# imprime as 5 primeiras linha dos dados montados
sonar.head()

## Transforma dados categóricos em números 

- M => 0 (mina)
- R => 1 (rocha)

In [ ]:
sonar.replace(('R', 'M'), (1, 0), inplace=True)
sonar.head()

## Separa os atributos das classes

In [ ]:
sonar = sonar.values
print(type(sonar))
X = sonar[:,0:60].astype(float) # X recebe todas as colunas exceto a última
y = sonar[:,60] # y recebe a última coluna
print(X.shape)
print(y.shape)

## Divide o conjunto de dados em treino e teste

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

## Normaliza os dados

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Criação do modelo de rede neural



In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import plot_model

model = Sequential()
# Camada oculta com 30 neurônios (definição das entradas)
model.add(Dense(30, input_dim=X_train.shape[1], activation='relu'))
# Dropout Regularization
model.add(Dropout(0.4))
# camada de saída com um neurônio
model.add(Dense(1, activation='sigmoid'))
# sumário (resumo) do modelo
model.summary()

## Compilação do modelo




In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Carrega o TensorBoard

In [ ]:
% load_ext tensorboard

### Define diretório de logs (registros) para o TensorBoard

In [ ]:
import datetime, os
logdir = os.path.join("logs", datetime.datetime.now().strftime("run_%Y_%m_%d-%H_%M_%S"))

## Treinamento do modelo



In [ ]:
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
tensorboard_callback = TensorBoard(logdir, histogram_freq = 1) # callback TensorBoard
history = model.fit(x=X_train,y=y_train, validation_data=(X_test,y_test), 
                    batch_size=5, epochs=80, 
                    callbacks=[early_stop,tensorboard_callback])

## Executa ferramenta de visualização TensorBoard

In [ ]:
% tensorboard --logdir logs

## Visualiza diretório de logs

In [ ]:
!ls -l ./logs

## Remove (limpa) diretório de logs

In [ ]:
!rm -rf ./logs

## Avaliação do modelo

In [ ]:
model.evaluate(X_test, y_test)